In [3]:
import openai
import pandas as pd
import random
from loguru import logger
from datetime import datetime

In [1]:
def init_openai ():
    openai.api_type = "azure"
    openai.api_base = "https://convaip-sbx-openai.openai.azure.com/"
    # openai.api_version = "2022-12-01" # For GPT3.0
    openai.api_version = "2023-03-15-preview" # For GPT 3.5
    openai.api_key = '9e6fa24631f54cf58866766bd31a2bff' #os.getenv("OPENAI_API_KEY")

In [3]:
def random_order(description: str) -> str:
    '''
    This function takes a sentence as input and changes the word order randomly. 
    '''
    # Split the description into individual words
    words = description.split()
    new_description = description

    # Shuffle the words in the description
    while new_description == description:
        random.shuffle(words)

        # Join the shuffled words back into a string
        new_description = ' '.join(words)

    return new_description

In [4]:
def swap_chars(description: str) -> str:
    '''
    This function takes a text as input and randomly swap two chars which are next to each other
    '''
    # Convert description to a list of chars
    chars = list(description)

    # Choose the chars which should be swaped 
    index1 = random.randrange(len(chars) - 1)
    index2 = index1 + random.choice([-1,1])

    # Change their position
    chars[index1], chars[index2] = chars[index2], chars[index1]
    
    return ''.join(chars)


In [5]:
def add_char(description: str) -> str:
    # German alphabet
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    
    # Randomly choose a char of the alphabet
    char = random.choice(alphabet)

    # Randomly select the position where to add the char
    index = random.randrange(len(description)+1)

    # Add the selected char
    new_description = description[:index] + char + description[index:]

    return new_description 

In [6]:
def delete_char(description: str) -> str:
    index = random.randrange(len(description))
    new_description = description[:index] + description[index+1:]
    
    return new_description   

In [7]:
def random_mistakes(description: str) -> str:
    '''
    This function takes a text as input and inserts a random spelling error. 
    This is done either by swapping chars, adding a chars, or deleting a chars.
    '''
    # Generate three random probabilities between 0 and 1 
    probs = [random.uniform(0,1) for _ in range(3)]
    probs = [p/sum(probs) for p in probs]

    if probs[0] > (probs[1] and probs[2]):
        # Randomly swap two chars
        new_description = swap_chars(description)
        
        return new_description
    
    elif probs[1] > (probs[0] and probs[2]):
        # Randomly add a char
        new_description = add_char(description)

        return new_description

    else:
        # Randomly delete one char
        new_description = delete_char(description)
        
        return new_description

In [8]:
def create_prompt(text: str) -> str:
    prompt = f'''
                Task:
                Write a modified car component designation based on the given German language input. Your output must be slightly different to the input while retaining the same meaning.

                Context:
                The original designation is written in German and may contain abbreviations and technical terms. You are not allowed to add or remove words. Abbreviations that often appear in the data set are 'HI' for 'HINTEN', 'VO' for 'VORN', 'HKL' for 'HECKKLAPPE','GPR' for GEPAECKRAUM', 'VKL' for 'VERKLEIDUNG', 'ISP' for 'INNENSPIEGEL', 'TV' for 'TUER VORNE', 'TH' for 'TUER HINTEN', 'HBL' for 'HOCHGESETZTE BREMSLEUCHTE', 'STF' for 'STOSSFAENGER', 'KST' for 'KOPFSTUETZE', 'ND' for 'NORMALDACH'. This abbreviation help you to create designations.

                Instructions:
                1. Carefully examine the input and understand its meaning.
                2. Modify the designation to create a new version which is very close to the original.
                3. Make sure that the meaning of the designation remains the same.
                4. Make sure that the modified designation is not equal to the input data

                Examples: 
                - Input Data: ABDECKLEISTE EINSTIEG HI
                - Modified Designation: ABDECKLEISTE EINSTIEG HINTEN

                - Original: KANTENSCHUTZ TUER VORN
                - Modified Designation: KANTENSCHUTZ TV

                - Original: MD KST VERSTELLBAR AUSSEN MAT
                - Modified Designation: MD KST AUßENMATTE VERSTELLBAR

                - Original: SEITENVERKLEIDUNG
                - Modified Designation: SEITENVERKL

                - Orignal: ABDECKUNG FENSTERRAHMEN TUER VORNE
                - Modified Designation: ABDECKUNG FENSTERRAHMEN TUERE VORN

                """
                {text}
                """
            '''
    return prompt

In [9]:
def remove_prefix(response: str) -> str:
    ''' 
    The output of GPT has always a prefix like "Answer: " or "Modified Designation: ". This prefix will be deleted that we keep only the generated text.
    '''
    index = response.find(":")

    if index != -1:
        new_response = response[index+1:].lstrip()
    else:
        new_response = response
    
    return new_response

In [10]:
def gpt30_designation(text:str) -> str:
    prompt= create_prompt(text)
    new_response = text
    while new_response == text:
        response = openai.Completion.create(
            engine="davinci-003-deployment",
            prompt=prompt,
            temperature=0.7,
            max_tokens=200,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            n=1
        )
        response = [choice.text.strip() for choice in response.choices]
        new_response = remove_prefix(response[0])
    return new_response

In [11]:
def gpt35_designation(text:str) -> str:
    prompt= create_prompt(text)
    new_response = text
    while new_response == text:
        response = openai.ChatCompletion.create(
            engine="chat-gpt-0301",
            messages=[{"role":"system","content":"You are an AI assistant that helps to create a car component designation."},{"role":"user","content": prompt}],
            temperature=0.6,
            max_tokens=200,
            top_p=1,
            n=1
        )
        response = [choice.message.content for choice in response.choices]
        new_response = remove_prefix(response[0])
    return new_response

In [4]:
def data_augmentation(df: pd.DataFrame, rand_order:bool, rand_mistakes:bool, gpt:bool, df_to_excel:bool) -> pd.DataFrame:
    ''' 
    This function generates synthetic data to extend the data set. 
    Only the data points that are relevant for a measurement are expanded, since this class is very underrepresented. 
    A component name is expanded if it contains more than 2 words and at least one of the three data augmentation techniques is activated.
    Return: New dataset 
    '''
    init_openai()

    logger.info("Start adding artificial designations...")

    df = df.drop_duplicates(subset=["Sachnummer"],ignore_index=True)

    # Create a new empty dataframe with the same columns as the original dataframe
    new_df = df.iloc[[0]]

    # Iterate over the rows in the original dataframe
    for i, row in df.iterrows():
        # Copy the orignal sample to a temporary dataframe
        temp_df = df.iloc[[i]]
        
        # Add the original sample to the new dataframe
        if i>0:
            new_df = pd.concat([new_df,temp_df], ignore_index=True)

        if row["Relevant fuer Messung"]=="Ja":
            if len(row["Benennung (dt)"].split()) > 2:
                if rand_order:
                    # Generate a new description by random ordering and adding the new sample to the dataset
                    temp_df.loc[i,"Benennung (dt)"] = random_order(row["Benennung (dt)"])
                    new_df = pd.concat([new_df, temp_df], ignore_index=True)

                if rand_mistakes:
                    # Create a new description by incorporating random mistakes and adding the new sample to the dataset
                    temp_df.loc[i,"Benennung (dt)"] = random_mistakes(row["Benennung (dt)"])
                    new_df = pd.concat([new_df, temp_df], ignore_index=True)
                    
                if gpt:
                    # Create a new description with GPT and adding the new sample to the dataset
                    temp_df.loc[i,"Benennung (dt)"] = gpt35_designation(row["Benennung (dt)"])
                    if len(temp_df["Benennung (dt)"][i]) < 40:
                        new_df = pd.concat([new_df, temp_df], ignore_index=True)
            else:
                continue
        else: 
            continue
    
    if df_to_excel == True:
        dateTimeObj = datetime.now()
        timestamp = dateTimeObj.strftime("%d%m%Y_%H%M")

        new_df.to_excel(f"../data/artificial_dataset_{timestamp}.xlsx")

    logger.success(f"Creating a new dataset with artificial designations was succeccfull!")

    return new_df

### MAIN

In [13]:
def main():
    # Define the path to the labeled dataset
    data = pd.read_excel("../data/G65_bounding_pp.xlsx",index_col=0)
    
    # Declare which data augmentation techniques should be used
    rand_order = True
    rand_mistakes = True
    gpt = True

    # Generate the new dataset
    new_data = data_augmentation(data,rand_order,rand_mistakes,gpt, save_as_excel=False)

In [14]:
if __name__ == "__main__":
    
    main()

2023-05-02 16:43:16.424 | INFO     | __main__:data_augmentation:9 - Start adding artificial designations...
2023-05-02 16:43:39.885 | SUCCESS  | __main__:data_augmentation:52 - Creating a new dataset with artificial designations was succeccfull!
